In [29]:
import requests
import time
import datetime
from datetime import datetime, timedelta

In [3]:
import pandas as pd

In [4]:
def sendMessage(para, mensaje):
    url = 'http://localhost:3001/lead'
    
    data = {
        "message": mensaje,
        "phone": para
    }
    headers = {
        'Content-Type':'application/json'
    }
    print(data)
    response = requests.post(url, json=data, headers=headers)
    time.sleep(60)
    return response

In [5]:
now = datetime.datetime.now()

In [6]:
mario = '59175380725'

In [69]:
df_molienda = pd.read_excel(r'C:/Documents/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/Molienda.xlsx')

In [114]:
def definir_trapiche(df):
    # extrae la hora
    df['solo_hora'] = df['hora'].str.split('T').str[1]
    # concatena fecha y hora
    df['fecha_hora'] = df['fecha'] + ' ' + df['solo_hora']
    # convierte la columna a tipo datetime
    df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])
    # obtiene la fecha y hora actual
    hora_actual = datetime.now()
    # calcula una hora antes
    una_hora_antes = hora_actual - timedelta(hours=3)
    # filtra los resgistros de la ultima hora
    df_ultima_hora = df[(df['fecha_hora'] >= una_hora_antes) & (df['fecha_hora'] <= hora_actual)]
    trapiches = list(set(df_ultima_hora['trapiche']))
    if len(trapiches) == 0:
        return 0
    elif 1 in trapiches and 2 in trapiches:
        return 3
    elif 1 in trapiches:
        return 1
    elif 2 in trapiches:
        return 2
    print('=================== ERROR =====================')

In [122]:
def calcular_datos():
    df_playa = pd.read_excel(r'C:/Documents/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/ReportePlaya.xlsx')
    df_molienda = pd.read_excel(r'C:/Documents/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/Molienda.xlsx')
    df_horarios = pd.read_excel(r'C:/Documents/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/Horarios.xlsx')
    
    df_res_molienda = pd.merge(df_molienda, df_horarios[['Hora', 'Orden_Hora']], left_on='hora2', right_on='Hora', how='left')
    
    #cantidad de paquetes
    #cantidad de caña disponible
    filtro = df_playa[(df_playa['status'] == 'PL') | (df_playa['status'] == 'IN')]
    cantidad_paquetes = filtro['cantPqt'].sum()
    cana_disponible = cantidad_paquetes * 14
    
    #promedio lleganda pq
    df_playa['Hora_Entera'] = df_playa['horaDocum'].str[11:13].astype(int)
    max_hora_ent = df_playa['Hora_Entera'].max() - 3
    filtered_df = df_playa[df_playa['Hora_Entera'] >= max_hora_ent]
    sum_cant_pqt = filtered_df['cantPqt'].sum()
    promedio_llegada_pq = sum_cant_pqt / 3
    
    #trapiches
    # trapiche1    210 tn/ha    15 paquetes
    # trapiche2    690 tn/ha    49 paquetes
    
    #horas molienda
    horas_molienda_t1 = cantidad_paquetes / 15
    horas_molienda_t2 = cantidad_paquetes / 49
    horas_molienda_total = cantidad_paquetes / (15 + 49)
    
    #total paquetes resto dia
    total_paquetes_resto_dia_t1 = promedio_llegada_pq * horas_molienda_t1
    total_paquetes_resto_dia_t2 = promedio_llegada_pq * horas_molienda_t2
    total_paquetes_resto_dia_total = promedio_llegada_pq * horas_molienda_total
    
    #toneladas
    toneladas = df_molienda['netWeight'].sum() / 1000
    
    #planificacion actual
    planificacion_actual_t1 = df_res_molienda['Orden_Hora'].max() * 210
    planificacion_actual_t2 = df_res_molienda['Orden_Hora'].max() * 690
    planificacion_actual_total = df_res_molienda['Orden_Hora'].max() * (210 + 690)
    
    #diferencia actual
    diferencia_actual_t1 = toneladas - planificacion_actual_t1
    diferencia_actual_t2 = toneladas - planificacion_actual_t2
    diferencia_actual_total = toneladas - planificacion_actual_total
    
    #orden hora
    orden_hora = 24 - df_res_molienda['Orden_Hora'].max()
    
    #toneladas promedio
    toneladas_prom = (df_molienda['netWeight'].sum() / 1000) / (24 - orden_hora)
    
    trapiches = definir_trapiche(df_molienda)
    
    if trapiches == 0:
        mensaje = f'''*REPORTE*
*⚙️ Trapiches:* Detenidos
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* ---
*🕰️Total horas abas.:* _
*⏳Tiempo espera:* _
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* ---
*🔻 Diferencia actual:* ---
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* _
*📊Molienda segun estimacion:* _'''
    
    elif trapiches == 1:
        mensaje = f'''*REPORTE*
*⚙️ Trapiches:* solo 01
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* {round(total_paquetes_resto_dia_t1,2)}
*🕰️Total horas abas.:* _
*⏳Tiempo espera:* _
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* {round(planificacion_actual_t1,2)}
*🔻 Diferencia actual:* {round(diferencia_actual_t1,2)}
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* _
*📊Molienda segun estimacion:* _'''
    
    elif trapiches == 2:
        mensaje = f'''*REPORTE*
*⚙️ Trapiches:* solo 02
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* {round(total_paquetes_resto_dia_t2,2)}
*🕰️Total horas abas.:* _
*⏳Tiempo espera:* _
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* {round(planificacion_actual_t2,2)}
*🔻 Diferencia actual:* {round(diferencia_actual_t2,2)}
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* _
*📊Molienda segun estimacion:* _'''
    
    elif trapiches == 3:
        mensaje = f'''*REPORTE*
*⚙️ Trapiches:* 01 y 02
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* {round(total_paquetes_resto_dia_total,2)}
*🕰️Total horas abas.:* _
*⏳Tiempo espera:* _
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* {round(planificacion_actual_total,2)}
*🔻 Diferencia actual:* {round(diferencia_actual_total,2)}
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* _
*📊Molienda segun estimacion:* _'''
        
    return mensaje

In [123]:
m = calcular_datos()

In [124]:
print(m)

*REPORTE*
*⚙️ Trapiches:* solo 01
*🚛 Paquetes disponibles:* 36
*🔢 Toneladas aprox.:* 504
*⏱️ Promedio llegada pq.:* 25.33
*📈 Paquetes estimados:* 60.8
*🕰️Total horas abas.:* _
*⏳Tiempo espera:* _
*🎋 Molienda actual:* 6608.19
*📅 Planificacion actual:* 4200
*🔻 Diferencia actual:* 2408.19
*🕒 Promedio horario:* 330.41
*🏭Molienda segun promedio:* _
*📊Molienda segun estimacion:* _


In [113]:
mensaje = f'''
*REPORTE*
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* {round(total_paquetes_resto_dia_total,2)}
*🕰️Total horas abas.:* no calculado 
*⏳Tiempo espera:* no calculado 
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* {round(planificacion_actual_total,2)}
*🔻 Diferencia actual:* {round(diferencia_actual_total,2)}
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* no calculado 
*📊Molienda segun estimacion:* no calculado 
'''

NameError: name 'cantidad_paquetes' is not defined

In [ ]:
while True:
    #now = datetime.datetime.now()
    cantidad_paquetes, cana_disponible = calcular_datos()
    mensaje = f'''
*Cantidad de paquetes en playa:* {cantidad_paquetes} paquetes.
*Cantidad de caña disponible:* {cana_disponible} toneladas.
'''
    sendMessage(me, mensaje)

{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
